In [4]:
import pandas as pd
import pickle

with open('cmps.pkl', 'rb') as f:
    cmps = pickle.load(f)

In [20]:
# three_sixty_path = "/home/msai/adnan002/data/open-data/data/three-sixty/"

# def get_360_data(match_id):
#     match_360_df = pd.read_json(three_sixty_path + '{}.json'.format(match_id))
#     match_360_df['match_id'] = match_id
#     return match_360_df

In [3]:
# all_360 = pd.DataFrame()

# for i, cmp in cmps.iterrows():
#     for match_id in cmp["match_ids"]:
#         # use pandas.concat to add df rows to all_360
#         try:
#             all_360 = pd.concat([all_360, get_360_data(match_id)])
#         except:
#             print("Error with match_id: {}".format(match_id))
#             continue

In [4]:
# # set event_uuid as index
# all_360.set_index('event_uuid', inplace=True)

In [5]:
# # save all_360 to pickle
# all_360.to_pickle('all_360.pkl')

In [5]:
# load all_360 from pickle
all_360 = pd.read_pickle('all_360.pkl')
len(all_360)

663646

In [6]:
# load features_with_label_df.pkl as df
with open('features_with_label_df.pkl', 'rb') as f:
    features_with_label_df = pickle.load(f)

ids = []
# print all match_id features_with_label_df
for i in features_with_label_df.index:
    if i not in all_360.index:
        ids.append(i)

# remove features_with_label_df rows with ids
features_with_label_df.drop(ids, inplace=True)

In [7]:
len(features_with_label_df)

957

In [8]:
# Add the 'freeze_frame' column from all_360 to features_with_label_df
features_with_label_df['freeze_frame'] = all_360['freeze_frame']
features_with_label_df['match_id'] = all_360['match_id']

In [9]:
len(features_with_label_df)

957

In [10]:
import numpy as np

def get_vertical_range(fouler_pos, dist_to_goal):
    x = fouler_pos[0]
    y = fouler_pos[1]

    range1 = [x, 120]
    range2 = [0, x]

    # calculate distance between foulder_pos and [120, 40]
    dist1 = np.linalg.norm(np.array([120, 40]) - np.array(fouler_pos))
    dist2 = np.linalg.norm(np.array([0, 40]) - np.array(fouler_pos))

    diff1 = abs(dist1 - dist_to_goal)
    diff2 = abs(dist2 - dist_to_goal)

    # find whether dist1 or dist2 matches dist_to_goal. a error of 0.25 is allowed.
    if diff1 < 0.0025:
        return range1
    elif diff2 < 0.0025:
        return range2
    else:
        if diff1 < diff2:
            return range1
        else:
            return range2

In [11]:
# add column 'vertical_range' to features_with_label_df
features_with_label_df['vertical_range'] = None

for i, row in features_with_label_df.iterrows():
    for player in row['freeze_frame']:
        if player['actor'] == True:
            fouler_pos = player['location']
            
    dist_to_goal = row['distance_to_goal']
    rangeee = get_vertical_range(fouler_pos, dist_to_goal)

    # add rangeee to features_with_label_df
    features_with_label_df.at[i, 'vertical_range'] = rangeee

In [12]:
def get_teammate_opponent_count(row, rangee):
    try:
        range_start = rangee[0]
        range_end = rangee[1]
    except:
        print ("error!!")
        print (row)
        print (rangee)
    
    teammate_count = 0
    opponent_count = 0

    keeper_visible = 0

    for player in row['freeze_frame']:
        if player['teammate']:
            if not player['keeper']:
                if not player['actor']:
                    if range_start <= player['location'][0] <= range_end:
                        teammate_count += 1
            else:
                keeper_visible = 1
        else:
            if not player['keeper']:
                if not player['actor']:
                    if range_start <= player['location'][0] <= range_end:
                        opponent_count += 1

    return teammate_count, opponent_count, keeper_visible


In [13]:
# add column 'teammate_count' and 'opponent_count' to features_with_label_df using get_teammate_opponent_count()
features_with_label_df['teammate_count'] = None
features_with_label_df['opponent_count'] = None
features_with_label_df['keeper_visible'] = None

for i, row in features_with_label_df.iterrows():
    rangee = row['vertical_range']
    teammate_count, opponent_count, keeper_visible = get_teammate_opponent_count(row, rangee)

    # add teammate_count and opponent_count to features_with_label_df
    features_with_label_df.at[i, 'teammate_count'] = teammate_count
    features_with_label_df.at[i, 'opponent_count'] = opponent_count
    features_with_label_df.at[i, 'keeper_visible'] = keeper_visible

In [14]:
# make columns teammate_count and opponent_count numeric
features_with_label_df['teammate_count'] = pd.to_numeric(features_with_label_df['teammate_count'])
features_with_label_df['opponent_count'] = pd.to_numeric(features_with_label_df['opponent_count'])
features_with_label_df['keeper_visible'] = pd.to_numeric(features_with_label_df['keeper_visible'])

In [15]:
# save features_with_label_df to pickle
features_with_label_df.to_pickle('features_with_label_df_with_counter.pkl')